In [1]:
from pathlib import Path
import re
import time
from datetime import datetime
from dateutil import relativedelta

import numpy as np
import pandas as pd
import toad

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from util import data_helper
from projects.home_credit import model_config
from util import woe_helper, bm_helper, lgbm_helper, report_helper

import lightgbm as lgb

%load_ext autoreload
%autoreload 2

In [27]:
df_sample = data_helper.Data.load('df_sample')
df_train = df_sample.loc[df_sample['sample_type'] == 'train']

In [49]:
model = data_helper.Data.load('bm_model')

In [54]:
model.get_importance()

,var,coef,std_err,z,pvalue,std_var,feature_importance
ft_app_ext_source_2,ft_app_ext_source_2,0.709874,0.02,46.56,0.00,0.55,0.11
ft_app_ext_source_3,ft_app_ext_source_3,0.622046,0.02,38.28,0.00,0.56,0.09
ft_app_credit_annuity_ratio,ft_app_credit_annuity_ratio,0.529326,0.03,20.83,0.00,0.32,0.05
ft_app_ext_source_1,ft_app_ext_source_1,0.446291,0.02,19.43,0.00,0.40,0.05
ft_app_credit_goods_ratio,ft_app_credit_goods_ratio,0.514120,0.03,17.69,0.00,0.26,0.04
ft_app_organization_type,ft_app_organization_type,0.404310,0.04,9.46,0.00,0.27,0.03
ft_bureau_max_debt_credit_ratio_2520d,ft_bureau_max_debt_credit_ratio_2520d,0.336429,0.03,10.69,0.00,0.37,0.03
ft_app_code_gender,ft_app_code_gender,0.624456,0.04,14.74,0.00,0.19,0.03
ft_instal_max_dpd_720d,ft_instal_max_dpd_720d,0.410487,0.04,10.79,0.00,0.27,0.03
ft_app_short_employment,ft_app_short_employment,0.498576,0.07,7.29,0.00,0.19,0.03
